# 2.2 Interfejs `Parcelable`

W tym przykładzie będziemy posiadać dwie aktywności, poprzez mechanizm `Intent` włączymy drugą aktywność oraz dzięki zaimplementowaniu interfejsu `Parcelable` przekażemy cały obiekt. Do `activity_main.xml` dodajemy jeden `Button` przez naciśnięcie którego otworzymy drugą aktywność.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/sendButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:text="SEND"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Następnie tworzymy drugą aktywność `SecondActivity.kt` i uzupełniamy jej layout w pliku `second_activity.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Przejdźmy do manifestu i określmy hierarchię aktywności

In [ ]:
        <activity
            android:name=".SecondActivity"
            android:parentActivityName=".MainActivity"
            android:label="Second Activity"
            android:exported="false" />

Następnie utwórzmy klasę której instancję przekażemy pomiędzy aktywnościami.

In [ ]:
class Properties(val a: Int, val b: Int, val c: String)

Aby być w stanie intancję takiej klasy przekazać do innej aktywności, musi ona implementować interfejs `Parcelable` (lub `Serializable` - więcej o różnicach poniżej).

In [ ]:
package pl.edu.uwr.pum.parcelableexamplekotlin

import android.os.Parcel
import android.os.Parcelable

class Properties(private val a: Int, private val b: Int, private val c: String?) : Parcelable {
    constructor(parcel: Parcel) : this(
        parcel.readInt(),
        parcel.readInt(),
        parcel.readString()
    ) {
    }

    override fun writeToParcel(parcel: Parcel, flags: Int) {
        parcel.writeInt(a)
        parcel.writeInt(b)
        parcel.writeString(c)
    }

    override fun describeContents(): Int {
        return 0
    }

    companion object CREATOR : Parcelable.Creator<Properties> {
        override fun createFromParcel(parcel: Parcel): Properties {
            return Properties(parcel)
        }

        override fun newArray(size: Int): Array<Properties?> {
            return arrayOfNulls(size)
        }
    }
}

Jakc widzimy musimy zaimplementować dwie metody, konstruktor oraz **companion object** `Creator`. Ponieważ obiekt towarzyszący jest sam w sobie jest obiektem więc nie ma żadnych ograniczeń - może rozszerzać klasy i implementować interfejsy. Jeżeli chcemy przekazać instancję klasy przez mechanizm `Intent` lub zachować w obiekcie `Bundle` w pierwszym kroku niejawnie zostanie wywołana metoda `writeToParcel`. Metoda ta opakowuje wszystkie właściwości klasy do obiektu `Parcel` wywołując metodę `write...` z odpowiednim typem. Przy odtwarzaniu instancji klasy `Properties` w pierwszym kroku wywoływana jest metoda `createFromParcel` znajdująca się w obiekcie towarzyszącym `Creator`. Metoda ta wywołuje drugorzędny konstruktor przyjmujący jako argument `Parcel`. Istotnym elementem tej implementacji jest konieczność zachowania kolejności przy przekazywaniu właściwości do i z `Parcel`. Przyjrzyjmy się konstruktorowi drugorzędnemu i metodzie `writeToParcel`.

In [ ]:
constructor(parcel: Parcel) : this(
    parcel.readInt(),
    parcel.readInt(),
    parcel.readString()
) {
}

override fun writeToParcel(parcel: Parcel, flags: Int) {
    parcel.writeInt(a)
    parcel.writeInt(b)
    parcel.writeString(c)
}

Pola `a`, `b` i `c` są zapisywane do `Parcel` i odczytywane w konstruktorze dokładnie w tej samej kolejności - jest to niezbędne do poprawnego działania - zweróćmy uwagę że nie posługujemy się tutaj żadnym kluczem, czy jakimś unikalnym identyfikartorem.

Ostatnią metodą jest `describeContent` - w interfejsie zdefiniowana jest stała `CONTENT_FILE_DESCRIPTION` która powinna zostać użyta w tej metodzie do zwrócenia maski bitowej - tutaj zwrócimy wynik wwywołania metody `hashCode`.

In [ ]:
override fun describeContents(): Int {
    return hashCode()
}

Przejdźmy do implementacji metody `onClick` przycisku na głównej aktyności. Tym razem zrobimy to nieco inaczej, przechodzimy do pliku `MainActivity.kt`. Do metody `onCreate` dodajemy

In [ ]:
findViewById<Button>(R.id.sendButton).setOnClickListener(){
            
}

Ustawiamy `onClickListener` przycisku i jako argument przekazywany jest obiekt o typie  interfejsu `OnClickListener` znajdującego się w klasie `View` - implementujemy jako **lambdę**. Klasa `View` jest główną klasą wszystkich elementów interfejsu użytkownika - jest rozszerzana przez wszystkie klasy.

Dodajmy implementację `onClick`

In [ ]:
findViewById<Button>(R.id.sendButton).setOnClickListener(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
    startActivity(intent)
}

Tworzymy klucz dla `Intent`, następnie tworzymy instancję `Properties` oraz sam `Intent`. Wskazujemy kontekst jako `this` oraz cel którym jest `SecoondActivity::class.java`. Następnie dodajemy za pomocą metody `putExtra` nasz obiekt `Properties` i wywołujemy `startActivity` w celu włączenia drugiej aktywności.

W następnym kroku odbierzemy dane w `SecondActivity` i rozpakujemy obiekt `Properties`. Przejdźmy do pliku `SecondActivity.kt`

In [ ]:
package pl.edu.uwr.pum.parcelableexamplekotlin

import androidx.appcompat.app.AppCompatActivity
import android.os.Bundle
import android.widget.TextView

class SecondActivity : AppCompatActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_second)
        val prop = intent.getParcelableExtra<Properties>(EXTRA_KEY)
        if (prop != null)
            ((prop.a + prop.b).toString() + " " + prop.c)
                .also { findViewById<TextView>(R.id.textView).text = it }
    }
}

Tworzymy `Intent` i wywoołujemy metodę `getIntent`, następnie tworzymy obiekt typu `Properties` - tutaj niejawnie zoostanie wywołany konstruktor z argumentem typu `Parcel`. Wyciągamy obiekt za pomocą metody `getParcelableExtra` jako argument podając klucz. Do odpowiednich pól możemy dostać się poprzez odpowiednie **gettery**. Możemy przetestować aplikację.

## **Interfejs `Serializable`**

Dodajmy do projektu kolejną klasę `SerializableProperties`, która implementuje interfejs `Serializable` i powtórzmy wszystkie czynności.

In [ ]:
import java.io.Serializable

class SerializableProperties (val a: Int, b: Int, c: String): Serializable

Interfejs `Serializable` jest **interfejsem znacznikowym** więc nie jest koonieczne implementowanie żadnych metod

W klasie `MainActivity.kt` dodajmy jeszcze jedno pole

In [ ]:
const val EXTRA_SERIALIZABLE = "serializable_example"

Następnie utworzymy instancję klasy `SerializableProperties` i dodamy ją do wcześniej utworzonego `Intent`

In [ ]:
findViewById<Button>(R.id.sendButton).setOnClickListener(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
        .putExtra(EXTRA_SERIALIZABLE, SerializableProperties(11, 11, "Serializable"))
    startActivity(intent)
}

Przejdźmy do `SecondActivity`, rozpakujmy nasz drugi obiekt i dodajmy jego pola do `TextView`.

In [ ]:
class SecondActivity : AppCompatActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_second)
        val prop = intent.getParcelableExtra<Properties>(EXTRA_KEY)
        val serProp = intent.getSerializableExtra(EXTRA_SERIALIZABLE) as SerializableProperties
        if (prop != null)
            ((prop.a + prop.b).toString() + " " + prop.c + "----" +
                    serProp.a + serProp.b + " " + serProp.c)
                .also { findViewById<TextView>(R.id.textView).text = it }
    }
}

Zwróćmy uwagę że przy wywołaniu metody `getSerializableExtra` dostajjjemy obiekt o typie interfejsu `Serializable` i potrszebujemy obiekt typu `SerializableProperties`. Ponieważ nasza klasa implementuje ten interfejs, możemy bezpiecznie rzutować na `SerializableProperties`.

Po uruchomieniu aplikacji powinniśmy dostać podobny wynik jak poprzednio. Widzimy że właściwie mogę zrobić dokładnie to samo przy pomocy dwóch interfejsów - `Parcelable` i `Serializable`; więc czym się różnią?
- `Parcelable` jest szybszy od `Serializable`
- `Serializable` jest interfejsem znacznikowym, więc implementacja jest szybsza
- `Serializable` tworzy wiele tymczasowych obiektów - jest niewydajny
- możemy przekazać całą tablicę obiektów za pomocą `Parcelable`
- w kotlinie mamy do dyspozycji adnotację `@Parcelize` dzięki której dostajemy domyślną implementację wszystkich metod interfejsu `Parcelable`

## **Adnotacja `@Parcelize`**

Dodajmy kolejną klasę do projektu i jak poprzednio przekażmy jej intstancję jako `Extra` to drugiej aktywności. W pierwszym kroku musimy przejść do pliku `build.gradle(Module)` i dodać plugin

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-parcelize'
}

Następnie możemy wykorzystać adnotację `@Parcelize`

In [ ]:
import android.os.Parcelable
import kotlinx.parcelize.Parcelize

@Parcelize
class ParcelizeProperties (val a: Int, val b: Int, val c: String) : Parcelable

Dokonujemy modyfikacji w pliku `MainActivity.kt`

In [ ]:
findViewById<Button>(R.id.sendButton).setOnClickListener(){
    val intent = Intent(this, SecondActivity::class.java)
        .putExtra(EXTRA_KEY, Properties(1, 2, "String"))
        .putExtra(EXTRA_SERIALIZABLE, SerializableProperties(11, 11, "Serializable"))
        .putExtra(EXTRA_PARCELIZE, ParcelizeProperties(0, 0, "Parcelize"))
    startActivity(intent)
}

Oraz pliku `SecondActivity.kt`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_second)
        
    val prop = intent.getParcelableExtra<Properties>(EXTRA_KEY)
    val serProp = intent.getSerializableExtra(EXTRA_SERIALIZABLE) as SerializableProperties
    val parProp = intent.getParcelableExtra<ParcelizeProperties>(EXTRA_PARCELIZE)
        
    if (prop != null && parProp != null)
        ((prop.a + prop.b).toString() + " " + prop.c + "----" +
            serProp.a + serProp.b + " " + serProp.c + "----\n" +
            ( parProp.a + parProp.b).toString() + " " + parProp.c)
        .also { findViewById<TextView>(R.id.textView).text = it }
}

Teraz możemy przetestowac aplikację. Widzimy że mamy możliwość automatycznej implementacji wszystkich niezbędnych elementów interfejsu `Parcelable`. Warunkiem jest posiadanie wszystkich pól w **konstruktorze głównym**.